In [ ]:
import nest_asyncio
import subprocess, time, json, urllib.request

nest_asyncio.apply()

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient  
from langchain.agents import create_agent
from langchain_deepseek import ChatDeepSeek
from langchain.agents import create_agent
import asyncio
from langgraph.graph import StateGraph, END


client = MultiServerMCPClient({
    # Local custom tool
    "JobTools": {
        "transport": "stdio",  # Local subprocess communication
        "command": "/home/tom/apps/cache/python-envs/ML/bin/python",
        "args": ["/home/tom/WD/langchain1.0/mcp-tools/wrapper.py"],
    },

    # Playwright MCP server (via npx)
    "playwright": {
        "transport": "stdio",
        "command": "npx",
        "args": ["@playwright/mcp@latest",],
    },
})



In [ ]:
# RUN WITH llama-server -m Holo1.5-3B.Q4_K_S.gguf --jinja
from langchain_openai import ChatOpenAI

# Connect to your local llama.cpp server
llm = ChatOpenAI(
    model="Holo1.5-3B.Q4_K_S",         
    api_key="EMPTY",                    
    base_url="http://127.0.0.1:8080/v1",
    temperature=0.7,
)

In [ ]:
tools = await client.get_tools()  
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END
from typing import List, Literal

tool_node = ToolNode(tools)

In [ ]:
tools

In [ ]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def call_tools(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
# initialize the workflow from StateGraph
workflow = StateGraph(MessagesState)

# add a node named LLM, with call_model function. This node uses an LLM to make decisions based on the input given
workflow.add_node("LLM", call_model)

# Our workflow starts with the LLM node
workflow.add_edge(START, "LLM")

# Add a tools node
workflow.add_node("tools", tool_node)

# Add a conditional edge from LLM to call_tools function. It can go tools node or end depending on the output of the LLM. 
workflow.add_conditional_edges("LLM", call_tools)

# tools node sends the information back to the LLM
workflow.add_edge("tools", "LLM")

agent = workflow.compile()

In [ ]:
async for chunk in agent.astream(
    {"messages": [("user", "go to indeed and search for ml engineering roles and apply to one using the tools you have, make cover letter and resume")]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()
